In [5]:
%classpath add jar ../../konduit.jar

In [17]:
%%bash
konduit stop tensorflow-mnist

Stopping konduit server 'tensorflow-mnist'
Application 'tensorflow-mnist' terminated with status 0



In [7]:
package ai.konduit;

import ai.konduit.serving.endpoint.Endpoint;

import io.vertx.core.Handler;
import io.vertx.core.http.HttpMethod;
import io.vertx.ext.web.RoutingContext;

import javax.imageio.ImageIO;
import java.awt.image.BufferedImage;
import java.io.File;
import java.io.IOException;
import java.util.ArrayList;
import java.util.Arrays;
import java.util.List;

import ai.konduit.serving.pipeline.api.pipeline.PipelineExecutor;

import io.micrometer.prometheus.PrometheusConfig;
import io.micrometer.prometheus.PrometheusMeterRegistry;
import io.micrometer.core.instrument.Counter;
import io.micrometer.core.instrument.binder.jvm.JvmMemoryMetrics;
import io.micrometer.core.instrument.binder.system.ProcessorMetrics;

import java.util.Timer;
import java.util.TimerTask;
import io.vertx.core.http.HttpHeaders;

public class PrometheusEndpoint implements Endpoint {

    public static PrometheusMeterRegistry registry;
    public static List<Counter> classCounterIncrement = new ArrayList();
    public static List<String> labels = Arrays.asList("Number_0", "Number_1", "Number_2", "Number_3", "Number_4", "Number_5", "Number_6", "Number_7", "Number_8", "Number_9");
    
    static {
        registry = new PrometheusMeterRegistry(PrometheusConfig.DEFAULT);
        
        if (registry != null) {
            System.out.println("Using metrics registry " + registry.getClass().getName() + " for inference");
            new JvmMemoryMetrics().bindTo(registry);
            new ProcessorMetrics().bindTo(registry);
            
            Counter serverUpTimeCounter = registry.counter("server.up.time");
            double increment = 5.0;
            new Timer().schedule(new TimerTask() {
                @Override
                public void run() {
                    serverUpTimeCounter.increment(increment);
                }
            }, 5000, ((int) increment) * 1000);
            
            
            for (String label : labels) {
                classCounterIncrement.add(Counter.builder(label)
                        .description("Classification counts seen so far for class label: " + label)
                        .baseUnit("classification.outcome")
                        .register(registry));
            }            
        } else {
            System.out.println("Not using metrics registry.");
        }
    }
    
    private PipelineExecutor pipelineExecutor;

    public PrometheusEndpoint(PipelineExecutor pipelineExecutor) { 
        this.pipelineExecutor = pipelineExecutor;
    }

    public HttpMethod type() { return HttpMethod.GET; }

    public String path() { return "/tensorflow-metrics"; }

    public List<String> consumes() { return Arrays.asList(); }

    public List<String> produces() { return Arrays.asList("text/plain; version=0.0.4; charset=utf-8"); }

    @Override
    public Handler<RoutingContext> handler() {
        return handler -> handler.response().putHeader(HttpHeaders.CONTENT_TYPE, "text/plain; version=0.0.4; charset=utf-8").end(registry.scrape());
    }
}

ai.konduit.PrometheusEndpoint

In [8]:
package ai.konduit;

import ai.konduit.serving.endpoint.Endpoint;
import ai.konduit.serving.pipeline.api.data.Data;
import ai.konduit.serving.pipeline.api.data.Image;
import ai.konduit.serving.pipeline.api.pipeline.Pipeline;
import ai.konduit.serving.pipeline.api.pipeline.PipelineExecutor;
import ai.konduit.serving.pipeline.impl.format.JavaImageFactory;
import ai.konduit.serving.pipeline.registry.ImageFactoryRegistry;
import io.vertx.core.Handler;
import io.vertx.core.http.HttpMethod;
import io.vertx.ext.web.RoutingContext;

import javax.imageio.ImageIO;
import java.awt.image.BufferedImage;
import java.io.File;
import java.io.IOException;
import java.util.ArrayList;
import java.util.Arrays;
import java.util.List;

import ai.konduit.serving.pipeline.util.ObjectMappers;
import ai.konduit.serving.pipeline.registry.NDArrayConverterRegistry;
import ai.konduit.serving.data.nd4j.format.ND4JConverters;

import io.micrometer.core.instrument.Counter;

import org.nd4j.linalg.api.ndarray.INDArray;
import org.nd4j.linalg.factory.Nd4j;

public class OCREndPoint implements Endpoint {

    private PipelineExecutor pipelineExecutor;

    private Counter requestsHandled = PrometheusEndpoint.registry.counter("requests.handled");
    
    public OCREndPoint(PipelineExecutor pipelineExecutor) { 
        this.pipelineExecutor = pipelineExecutor; 
        ImageFactoryRegistry.addFactory(new JavaImageFactory()); 
        NDArrayConverterRegistry.addConverter(new ND4JConverters.Nd4jToSerializedConverter()); 
        NDArrayConverterRegistry.addConverter(new ND4JConverters.SerializedToNd4jArrConverter());
    }

    public HttpMethod type() { return HttpMethod.POST; }

    public String path() { return "/infer"; }

    public List<String> consumes() { return Arrays.asList("application/octet-stream","multipart/form-data"); }

    public List<String> produces() { return Arrays.asList("application/json"); }

    @Override
    public Handler<RoutingContext> handler() {
        return handler -> {
            handler.vertx().executeBlocking(taskHandler -> {
                Data image = Data.empty();
                
                try {
                    image.put("image",Image.create(ImageIO.read(new File(handler.fileUploads().iterator().next().uploadedFileName()))));
                } catch (IOException e) {
                    e.printStackTrace();
                }

                Data exec = pipelineExecutor.exec(image);
                
                INDArray output = exec.getNDArray("output_layer/Softmax").getAs(INDArray.class);
                handler.response().end(ObjectMappers.toJson(output.data().asFloat()));
                taskHandler.complete();
                
                requestsHandled.increment();
                
                INDArray argMax = Nd4j.argMax(output, -1);
                for(int i = 0; i < argMax.length(); i++) {
                    PrometheusEndpoint.classCounterIncrement.get(argMax.getInt(i)).increment();
                }
            },resultHandler -> {
                if(resultHandler.failed()) {
                    if(resultHandler.cause() != null)
                        if(handler.vertx().exceptionHandler() != null)
                            handler.vertx().exceptionHandler().handle(resultHandler.cause());
                        else {
                            resultHandler.cause().printStackTrace();
                        }
                    else {
                        System.err.println("Failed to process classification endpoint async task. Unknown cause.");
                    }
                }
            });

        };
    }
}

ai.konduit.OCREndPoint

In [9]:
package ai.konduit;

import ai.konduit.serving.endpoint.Endpoint;
import ai.konduit.serving.endpoint.HttpEndpoints;
import ai.konduit.serving.pipeline.api.pipeline.Pipeline;
import ai.konduit.serving.pipeline.api.pipeline.PipelineExecutor;

import java.util.Arrays;
import java.util.List;

public class OCREndPoints implements HttpEndpoints {
    
    @Override
    public List<Endpoint> endpoints(Pipeline pipeline, PipelineExecutor pipelineExecutor) {
        return Arrays.asList(new OCREndPoint(pipelineExecutor), new PrometheusEndpoint(pipelineExecutor));
    }
}

ai.konduit.OCREndPoints

In [10]:
import java.net.URLClassLoader;
import java.net.URL;
import java.io.File;

import java.util.ArrayList;
import java.util.List;

import org.apache.commons.io.FileUtils;
import java.io.IOException;

import java.nio.charset.StandardCharsets;

URL[] urls = ((URLClassLoader) Class.forName("ai.konduit.serving.vertx.config.InferenceConfiguration").getClassLoader()).getURLs();
List<String> classpaths = new ArrayList<>();

for(URL url : urls) {
    String singleClassPath = new File(url.toURI()).getAbsolutePath();
    System.out.println(singleClassPath);
    classpaths.add(singleClassPath);
}

try {
    String output = String.join(File.pathSeparator, classpaths);
    File classpathOutputPath = new File("classpath");
    FileUtils.writeStringToFile(new File("classpath"), output, StandardCharsets.UTF_8);
    System.out.format("Saved %s at: %s%n", output, classpathOutputPath.getAbsolutePath());
} catch (IOException e) {
    e.printStackTrace();
}

/tmp/beaker7421797752502239521/outDir
/home/shams/PycharmProjects/konduit-serving-demo/konduit.jar
Saved /tmp/beaker7421797752502239521/outDir:/home/shams/PycharmProjects/konduit-serving-demo/konduit.jar at: /home/shams/PycharmProjects/konduit-serving-demo/demos/4-tensorflow-mnist/classpath


null

In [11]:
%%bash
java -cp $(cat classpath) ai.konduit.serving.cli.launcher.KonduitServingLauncher serve -id tensorflow-mnist -c tensorflow.json -rwm -b

Starting konduit server...
Using classpath: /tmp/beaker7421797752502239521/outDir:/home/shams/PycharmProjects/konduit-serving-demo/konduit.jar
INFO: Running command /home/shams/miniconda3/envs/beakerx/jre/bin/java -Dkonduit.logs.file.path=/home/shams/.konduit-serving/command_logs/tensorflow-mnist.log -Dlogback.configurationFile=/tmp/logback-run_command_819daf12dc5f49ae.xml ai.konduit.serving.cli.launcher.KonduitServingLauncher run --instances 1 -s inference -c tensorflow.json -Dserving.id=tensorflow-mnist
For server status, execute: 'java ai.konduit.serving.cli.launcher.KonduitServingLauncher list'
For logs, execute: 'java ai.konduit.serving.cli.launcher.KonduitServingLauncher logs tensorflow-mnist'



In [13]:
%%bash
konduit logs tensorflow-mnist -l 1000

11:21:22.406 [main] INFO  a.k.s.c.l.command.KonduitRunCommand - Processing configuration: /home/shams/PycharmProjects/konduit-serving-demo/demos/4-tensorflow-mnist/tensorflow.json
11:21:22.412 [main] INFO  u.o.l.s.context.SysOutOverSLF4J - Replaced standard System.out and System.err PrintStreams with SLF4JPrintStreams
11:21:22.414 [main] INFO  u.o.l.s.context.SysOutOverSLF4J - Redirected System.out and System.err to SLF4J for this context
11:21:22.414 [main] INFO  a.k.s.c.l.command.KonduitRunCommand - Starting konduit server with an id of 'tensorflow-mnist'
11:21:22.734 [vert.x-worker-thread-0] INFO  a.k.s.p.registry.PipelineRegistry - Loaded 28 PipelineStepRunnerFactory instances
11:21:26.414 [vert.x-worker-thread-0] INFO  a.k.s.v.verticle.InferenceVerticle - 

####################################################################
#                                                                  #
#    |  /   _ \   \ |  _ \  |  | _ _| __ __|    |  /     |  /      #
#    . <   (   | .  

In [14]:
%%bash
konduit list


Listing konduit servers...

 #   | ID                             | TYPE       | URL                  | PID     | STATUS     
 1   | tensorflow-mnist               | inference  | 0.0.0.0:9008         | 12129   | started    




In [15]:
%%html
  <div id="banner">
    <div style="display: inline-block">
        <img src="test_files/test_input_number_0.png"/>
    </div>

    <div style="display: inline-block">
        <img src="test_files/test_input_number_1.png"/>
    </div>

    <div style="display: inline-block">
        <img src="test_files/test_input_number_2.png"/>
    </div>
      
    <div style="display: inline-block">
        <img src="test_files/test_input_number_3.png"/>
    </div>
      
    <div style="display: inline-block">
        <img src="test_files/test_input_number_4.png"/>
    </div>
      
    <div style="display: inline-block">
        <img src="test_files/test_input_number_5.png"/>
    </div>

    <div style="display: inline-block">
        <img src="test_files/test_input_number_6.png"/>
    </div>

    <div style="display: inline-block">
        <img src="test_files/test_input_number_7.png"/>
    </div>
      
    <div style="display: inline-block">
        <img src="test_files/test_input_number_8.png"/>
    </div>
      
    <div style="display: inline-block">
        <img src="test_files/test_input_number_9.png"/>
    </div>
      
</div>

In [16]:
import io.vertx.core.*;
import io.vertx.ext.web.client.WebClient;
import java.util.concurrent.TimeUnit;
import io.vertx.ext.web.multipart.MultipartForm;

Vertx vertx = Vertx.vertx(new VertxOptions().setMaxEventLoopExecuteTime(10).setMaxEventLoopExecuteTimeUnit(TimeUnit.SECONDS));
    
WebClient client = WebClient.create(vertx);

for(int j = 0; j < 10000; j++) {
    for (int k = 0; k < 10; k++) {
        MultipartForm form = MultipartForm.create()
                .attribute("imageDescription", "Image input" + k)
                .binaryFileUpload("image", "test_input_number_" + k + ".png", "test_files/test_input_number_" + k + ".png", "image/png");

        // Submit the form as a multipart form body
        client.post(9008, "localhost", "/infer")
            .sendMultipartForm(form, ar -> {});
    }
}

null

## Metrics
Visit: http://localhost:3000/d/lP_JcnHWz-local/pipeline-metrics?orgId=1&refresh=5s to view metrics